<a href="https://colab.research.google.com/github/chuckyLeeVIII/Bitcoin-BhE-NaS/blob/master/IDEX_Hollow_Key_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import hashlib
from typing import List, Tuple, Dict, Any
from random import SystemRandom
from datetime import datetime
import json

# --- DEPENDENCY: NoS TSS Engine (Previously Defined) ---

class NoSTssEngine:
    """
    The CAD-0 NoS Protocol TSS Engine Dependency. (2-of-3) SSS logic.
    """
    PRIME = 2**256 - 2**32 - 977
    KEY_SIZE_BITS = 256
    QUORUM_M = 2
    TOTAL_N = 3

    def __init__(self):
        self.rng = SystemRandom()

    @staticmethod
    def _modular_inverse(a: int, m: int) -> int:
        """Computes the modular multiplicative inverse a^-1 mod m using pow(a, m-2, m)."""
        if m == 0: raise ValueError("Modulus cannot be zero.")
        return pow(a, m - 2, m)

    def create_master_key_and_shares(self) -> Tuple[int, List[Tuple[int, int]], List[int]]:
        """
        Generates the master secret key (S), N shares, and the polynomial coefficients.
        Returns: (Master Secret S, Shares [(x,y), ...], Coefficients [a0, a1, ...])
        """
        S = self.rng.getrandbits(self.KEY_SIZE_BITS) % self.PRIME
        coefficients = [S]
        for _ in range(self.QUORUM_M - 1):
            coefficients.append(self.rng.getrandbits(self.KEY_SIZE_BITS) % self.PRIME)

        def P(x):
            y = 0
            for i, a_i in enumerate(coefficients):
                y = (y + a_i * (x ** i)) % self.PRIME
            return y

        shares = [(x_i, P(x_i)) for x_i in range(1, self.TOTAL_N + 1)]
        return S, shares, coefficients

    def reconstruct_master_key(self, required_shares: List[Tuple[int, int]]) -> int:
        # Implementation omitted for brevity, assumes functional Lagrange Interpolation
        pass

# --- 2. IDEX GENERATOR (Hollow Key Minting Utility) ---

class HollowKeyGenerator:
    """
    The IDEX SBT Index Generator for the Hollow Key.
    Ensures deterministic, unique ID generation based on the TSS setup.
    """
    def __init__(self, tss_engine: NoSTssEngine):
        self.engine = tss_engine

    def _generate_public_parameters_string(self, coefficients: List[int], shares: List[Tuple[int, int]]) -> str:
        """
        Creates a canonical, deterministic string from the public TSS parameters.

        The Hollow Key's ID must be derived only from PUBLIC information to prevent
        leaking the Master Secret S. The unique hash serves as the verifiable ID.
        """
        public_data = {
            "prime": self.engine.PRIME,
            "quorum": self.engine.QUORUM_M,
            # Use public data points (x_i, y_i) from all shares
            "shares_public": sorted(shares),
            # Use the polynomial coefficients (public info derived from the shares)
            "poly_coeffs_commitment": [hashlib.sha256(str(c).encode()).hexdigest() for c in coefficients]
        }
        # Canonical JSON string ensures hash deterministicity
        return json.dumps(public_data, sort_keys=True, separators=(',', ':'))

    def generate_hollow_key(self, issuer_address: str, user_public_key: str) -> Dict[str, Any]:
        """
        Mints the Hollow Key SBT (IDEX generator function).

        1. Executes the DKG/TSS key generation ceremony.
        2. Generates the unique, deterministic ID (idx).
        3. Creates the zk-STARK commitment hash for the on-chain anchor.

        :param issuer_address: The BTC-Branch wallet address issuing the key.
        :param user_public_key: The user's primary wallet public key (the "Soul").
        :return: Hollow Key Metadata structure ready for anchoring.
        """

        # --- STEP 1: Execute Distributed Key Generation (DKG) ---
        # The Master Key S is generated and shares S1, S2, S3 are created.
        master_secret, shares, coefficients = self.engine.create_master_key_and_shares()

        # --- STEP 2: Generate Canonical Public ID String ---
        canonical_string = self._generate_public_parameters_string(coefficients, shares)

        # --- STEP 3: Generate Hollow Key ID (idx) ---
        # Use SHA3-256 for superior collision resistance (keccak_256)
        h = hashlib.sha3_256()
        h.update(canonical_string.encode('utf-8'))
        h.update(user_public_key.encode('utf-8')) # Bind key to user's root identity

        hollow_key_idx = h.hexdigest()

        # --- STEP 4: Generate zk-STARK Commitment (Anchor) ---
        # This hash commits to the entire security setup configuration.
        # In the final system, this is the hash of the STARK Prover's public input/transcript.

        # PoC: Hash the final ID and the creation time for finality commitment.
        final_commitment_input = f"{hollow_key_idx}:{datetime.now().timestamp()}:{issuer_address}"
        zk_stark_commitment = hashlib.sha256(final_commitment_input.encode('utf-8')).hexdigest()

        # --- STEP 5: Package Hollow Key Metadata ---
        hollow_key_metadata = {
            "sbt_idx": hollow_key_idx,
            "type": "HollowKey",
            "protocol": "CAD-0 NoS Protocol",
            "issuer": issuer_address,
            "owner_soul_address": user_public_key,
            "creation_timestamp": datetime.now().isoformat(),
            "master_key_commitment": zk_stark_commitment,
            "tss_setup": {
                "quorum_m_of_n": f"{self.engine.QUORUM_M}-of-{self.engine.TOTAL_N}",
                "public_shares_anchors": {
                    "mobile_wallet_s1": f"x={shares[0][0]}, y={shares[0][1]}",
                    "home_node_s2": f"x={shares[1][0]}, y={shares[1][1]}",
                    "balancer_s3": f"x={shares[2][0]}, y={shares[2][1]}",
                }
            },
            # WARNING: Master Secret and Shares MUST be delivered securely OFF-CHAIN.
            # They are only returned here for PoC demonstration purposes.
            "_poc_master_secret": master_secret,
            "_poc_shares": shares
        }

        # The shares S1, S2, S3 must be securely encrypted and delivered to the
        # respective hardware devices (Layer 0) for binding via the NoS protocol.

        return hollow_key_metadata

# --- EXECUTION EXAMPLE (IDEX GENERATION) ---

if __name__ == '__main__':
    # Initialize the core engine and the generator utility
    tss_engine = NoSTssEngine()
    idx_generator = HollowKeyGenerator(tss_engine)

    # Mock user data for the IDEX generation ceremony
    MOCK_ISSUER = "BTC_Branch_Contract_12345"
    MOCK_USER_PUBKEY = "HollowKeySoulAddress_0xABCDEF0123456789"

    print("--- IDEX SBT GENERATOR: HOLLOW KEY MINTING CEREMONY (CAD-0) ---")

    # Generate the first Hollow Key
    hollow_key_1 = idx_generator.generate_hollow_key(MOCK_ISSUER, MOCK_USER_PUBKEY)

    print("\n[SUCCESS] HOLLOW KEY IDEX GENERATED:")
    print(f"IDEX: {hollow_key_1['sbt_idx']}")
    print(f"COMM: {hollow_key_1['master_key_commitment']}")
    print("-" * 50)
    print(json.dumps(hollow_key_1, indent=4))

    # CRITICAL: Verify determinism/uniqueness (A new generation should have a new IDEX)
    # Note: Since the master_secret is randomly generated (Step 1), the IDEX will change
    # with each run, ensuring a unique ID for every individual TSS setup ceremony.

    print("\n[VERIFICATION] Hollow Key Shares Generated (Delivered to Hardware):")
    print(f"Master Secret (S): {hollow_key_1['_poc_master_secret']}")
    print(f"Mobile Wallet Share: {hollow_key_1['_poc_shares'][0]}")
    print(f"Home Node Share: {hollow_key_1['_poc_shares'][1]}")